In [1]:
import pandas as pd
import numpy as np
import os
import re
from PIL import Image
import h5py
from sklearn.model_selection import train_test_split
import argparse

In [2]:
import copy
import logging
import torch
import torch.nn as nn
from torchvision import *
import random
import sys
import time
import cv2

In [3]:
img_size = 512

val_output_path = 'val.h5'
img_dir =  r'ISIC2018_Task1-2_Validation_Input/'
mask_dir = 'ISIC2018_Task2_Validation_GroundTruth/'

IMG_FILE_REGEX = r'ISIC_(\d+).jpg'
MASK_FILE_REGEX = r'ISIC_(\d+)_attribute_(.*).png'

ISIC2019_FILE_REGEX = r'ISIC_(\d+).jpg'

ATTR_TO_INDEX = {
    'globules' : 0,
    'milia_like_cyst' : 1,
    'negative_network' : 2,
    'pigment_network' : 3,
    'streaks' : 4,
}

In [4]:
Back = '../../../../../594A/Dataset/ISIC challenge 2018 Task 1-2/'

img_dir =  Back + img_dir
mask_dir = Back + mask_dir

#  ISIC2018 Val

In [5]:
df_ISIC2018 = pd.DataFrame(columns=['image', 'globules', 'milia_like_cyst', 
                                    'negative_network', 'pigment_network', 'streaks'])

In [6]:
df_ISIC2018.head()

,image,globules,milia_like_cyst,negative_network,pigment_network,streaks


In [7]:
for img_file in os.listdir( img_dir ):
    if not img_file.endswith('.jpg'): continue
    img_id = str( re.search( IMG_FILE_REGEX, img_file ).group(1) )
    img_name = 'ISIC_' + img_id
    df_append = pd.DataFrame.from_records([{'image': img_name, 'globules': 2, 'milia_like_cyst': 2, 
                                      'negative_network': 2, 'pigment_network': 2, 'streaks': 2, 'pigment_network_pred': 2}])
    
    df_ISIC2018 = pd.concat([df_ISIC2018, df_append], ignore_index=True)

In [8]:
df_ISIC2018

,image,globules,milia_like_cyst,negative_network,pigment_network,streaks,pigment_network_pred
0,ISIC_0022221,2,2,2,2,2,2.0
1,ISIC_0036328,2,2,2,2,2,2.0
2,ISIC_0021914,2,2,2,2,2,2.0
3,ISIC_0036321,2,2,2,2,2,2.0
4,ISIC_0022192,2,2,2,2,2,2.0
...,...,...,...,...,...,...,...
95,ISIC_0018521,2,2,2,2,2,2.0
96,ISIC_0012255,2,2,2,2,2,2.0
97,ISIC_0023508,2,2,2,2,2,2.0
98,ISIC_0012633,2,2,2,2,2,2.0


## 2. save binary input 

In [9]:
'''
for mask_file in os.listdir( mask_dir ):
    if not mask_file.endswith('.png'): continue
    mask = Image.open( os.path.join( mask_dir, mask_file ) )
    mask = mask.resize( (img_size, img_size) )
    mask = np.array( mask )
    
    assert mask.shape == (img_size, img_size)
    assert mask.max() <= 255
    
    # decide feature exist or not
    if mask.max() > 0:
        binary_label = 1
    else:
        binary_label = 0
    
    m = re.search(MASK_FILE_REGEX, mask_file)
    
    file_idx = df_ISIC2018[df_ISIC2018['image'].str.contains(m.group(1))].index[0]
    
    if m.group(2) ==  'globules':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('globules')] = binary_label
    elif m.group(2) ==  'milia_like_cyst':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('milia_like_cyst')]  = binary_label
    elif m.group(2) ==  'negative_network':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('negative_network')]  = binary_label
    elif m.group(2) ==  'pigment_network':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('pigment_network')]  = binary_label
    elif m.group(2) ==  'streaks':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('streaks')]  = binary_label
'''

"\nfor mask_file in os.listdir( mask_dir ):\n    if not mask_file.endswith('.png'): continue\n    mask = Image.open( os.path.join( mask_dir, mask_file ) )\n    mask = mask.resize( (img_size, img_size) )\n    mask = np.array( mask )\n    \n    assert mask.shape == (img_size, img_size)\n    assert mask.max() <= 255\n    \n    # decide feature exist or not\n    if mask.max() > 0:\n        binary_label = 1\n    else:\n        binary_label = 0\n    \n    m = re.search(MASK_FILE_REGEX, mask_file)\n    \n    file_idx = df_ISIC2018[df_ISIC2018['image'].str.contains(m.group(1))].index[0]\n    \n    if m.group(2) ==  'globules':\n        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('globules')] = binary_label\n    elif m.group(2) ==  'milia_like_cyst':\n        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('milia_like_cyst')]  = binary_label\n    elif m.group(2) ==  'negative_network':\n        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('negative_network')]  = bina

In [10]:
#df_ISIC2018.to_csv('df_ISIC2018_val.csv', index=False)
df_ISIC2018 = pd.read_csv('dataframe_ISIC2018_val.csv')

# Train.h5py

In [11]:
with h5py.File(val_output_path, "w") as f:
    count = len(df_ISIC2018)

    # store imgs to h5 file
    images = f.create_dataset( 'images', (count, 3, img_size, img_size), dtype=np.uint8 )
    image_ids = f.create_dataset( 'image_ids', (count,), dtype=int )
    img_id_to_h5idx = {}
    i = 0
    for idx in range(len(df_ISIC2018)):
        print( f'Processing image {i+1}/{count}' )
        img_file = df_ISIC2018.iloc[idx].image + '.jpg'
        img = Image.open( os.path.join( img_dir, img_file ) )
        img = img.resize( (img_size, img_size) )
        img = np.array( img )
        img = img.transpose(2, 0, 1)
        #(3,512,1024)
        assert img.shape == (3, img_size, img_size)
        assert img.max() <= 255
        images[i] = img
        img_id = int( re.search( IMG_FILE_REGEX, img_file ).group(1) )
        image_ids[i] = img_id
        img_id_to_h5idx[ img_id ] = i
        i += 1
        
    # store masks to h5 file
    i = 0
    masks = f.create_dataset('masks', (count, 5, img_size, img_size), dtype=np.uint8)
    attr_labels = f.create_dataset('labels', (count, 5), dtype=np.uint8)
    for mask_file in os.listdir( mask_dir ):
        if not mask_file.endswith('.png'): continue
        print( f'Processing mask {i+1}/{5*count}' )
        mask = Image.open( os.path.join( mask_dir, mask_file ) )
        mask = mask.resize( (img_size, img_size) )
        mask = np.array( mask )
        assert mask.shape == (img_size, img_size)
        assert mask.max() <= 255
        m = re.search(MASK_FILE_REGEX, mask_file)
        img_id, attr_id = int(m.group(1)), ATTR_TO_INDEX[ m.group(2) ]
        
        
        h5idx = img_id_to_h5idx[ img_id ]
        masks[ h5idx, attr_id ] = mask 
        if mask.max() > 0:
            attr_labels[h5idx, attr_id] = 1
        i += 1      

Processing image 1/100
Processing image 2/100
Processing image 3/100
Processing image 4/100
Processing image 5/100
Processing image 6/100
Processing image 7/100
Processing image 8/100
Processing image 9/100
Processing image 10/100
Processing image 11/100
Processing image 12/100
Processing image 13/100
Processing image 14/100
Processing image 15/100
Processing image 16/100
Processing image 17/100
Processing image 18/100
Processing image 19/100
Processing image 20/100
Processing image 21/100
Processing image 22/100
Processing image 23/100
Processing image 24/100
Processing image 25/100
Processing image 26/100
Processing image 27/100
Processing image 28/100
Processing image 29/100
Processing image 30/100
Processing image 31/100
Processing image 32/100
Processing image 33/100
Processing image 34/100
Processing image 35/100
Processing image 36/100
Processing image 37/100
Processing image 38/100
Processing image 39/100
Processing image 40/100
Processing image 41/100
Processing image 42/100
P

Processing mask 252/500
Processing mask 253/500
Processing mask 254/500
Processing mask 255/500
Processing mask 256/500
Processing mask 257/500
Processing mask 258/500
Processing mask 259/500
Processing mask 260/500
Processing mask 261/500
Processing mask 262/500
Processing mask 263/500
Processing mask 264/500
Processing mask 265/500
Processing mask 266/500
Processing mask 267/500
Processing mask 268/500
Processing mask 269/500
Processing mask 270/500
Processing mask 271/500
Processing mask 272/500
Processing mask 273/500
Processing mask 274/500
Processing mask 275/500
Processing mask 276/500
Processing mask 277/500
Processing mask 278/500
Processing mask 279/500
Processing mask 280/500
Processing mask 281/500
Processing mask 282/500
Processing mask 283/500
Processing mask 284/500
Processing mask 285/500
Processing mask 286/500
Processing mask 287/500
Processing mask 288/500
Processing mask 289/500
Processing mask 290/500
Processing mask 291/500
Processing mask 292/500
Processing mask 